In [1]:
from pyrates import CircuitTemplate
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import cauchy
import yaml
# from rectipy import Network, random_connectivity
plt.rcParams['backend'] = 'TkAgg'

## Define distribution and parameters

In [2]:
def lorentzian(n: int, eta: float, delta: float, lb: float, ub: float):
    samples = np.zeros((n,))
    for i in range(n):
        s = cauchy.rvs(loc=eta, scale=delta)
        while s <= lb or s >= ub:
            s = cauchy.rvs(loc=eta, scale=delta)
        samples[i] = s
    return samples

In [3]:
# define parameters
###################

# model parameters
N = 1000
p = 0.2
C = 100.0
k = 0.7
v_r = -70.6
v_t = -50.4
Delta = 2.5
eta = 0.0
a = 0.03
b = 80
d = 10.0
g = 15.0
E_r = 0.0
tau_s = 6.0
v_spike = 1000.0
v_reset = -1000.0
# Add additional parameters for the AdEx model
C_adex = 281.0
delta_T = 2.0
g_adex = 30.0 
a_adex = 4.0
v_r_adex = -70.6
b_adex = 80.5
tau_w = 144.0

# define inputs
T = 500.0
# cutoff = 500.0
dt = 1e-3
dts = 1e-1
inp = np.ones((int(T/dt), 1))
# inp = np.zeros((int(T/dt), 1)) + 25.0
# inp[:int(cutoff*0.5/dt), 0] -= 15.0
# inp[int(750/dt):int(2000/dt), 0] += 30


In [4]:
# # Plot the input over time
# time = np.arange(0, T, dt)
# plt.figure(figsize=(10, 6))
# plt.plot(time, inp, label='Input')
# plt.xlabel('Time (ms)')
# plt.ylabel('Input')
# plt.title('Input over Time')
# plt.legend()
# plt.show()

## Running the Models

In [5]:
## AdEx model

# initialize model
adex = CircuitTemplate.from_yaml("/Users/utilizator/Desktop/THESIS CODE/gast_paper/config/adex_mf/adex")

# update parameters --- TODO!
adex.update_var(node_vars={'p/adex_op/C': C_adex,  'p/adex_op/v_r': v_r_adex, 'p/adex_op/v_t': v_t, 'p/adex_op/delta_T': delta_T,
                         'p/adex_op/a': a_adex, 'p/adex_op/b': b_adex, 'p/adex_op/tau_s': tau_s, 'p/adex_op/g': g_adex,
                         'p/adex_op/E_r': E_r, 'p/adex_op/tau_w': tau_w, 'p/adex_op/delta_v': Delta})

# run simulation
res_mf_adex = adex.run(simulation_time=T, step_size=dt, sampling_step_size=dts, solver='euler',
                outputs={'s': 'p/adex_op/s'}, inputs={'p/adex_op/I_ext': inp[:, 0]})


## just the AdEx model
t = res_mf_adex.index
# print(t, res_mf_adex['s'])

plt.figure(figsize=(10, 6))
plt.plot(t, res_mf_adex['s'], label='AdEx Model')
plt.xlabel('Time (ms)')
plt.ylabel('synaptic activity')
plt.title('Mean-Field Model Results')
plt.legend()
plt.show()

Compilation Progress
--------------------
	(1) Translating the circuit template into a networkx graph representation...
		...finished.
	(2) Preprocessing edge transmission operations...
		...finished.
	(3) Parsing the model equations into a compute graph...
		...finished.
	Model compilation was finished.
Simulation Progress
-------------------
	 (1) Generating the network run function...
	 (2) Processing output variables...
		...finished.
	 (3) Running the simulation...


/Users/utilizator/Desktop/THESIS CODE/gast_paper/pyrates_run.py:16: RuntimeWarning: overflow encountered in multiply
  dy[1:2] = (I_ext + g*(delta_T - s*(E_r + r*v) + 2*v + v_r + v_t) - w)/C
/Users/utilizator/Desktop/THESIS CODE/gast_paper/pyrates_run.py:16: RuntimeWarning: invalid value encountered in add
  dy[1:2] = (I_ext + g*(delta_T - s*(E_r + r*v) + 2*v + v_r + v_t) - w)/C


		...finished after 13.901698020000001s.


2024-11-04 11:32:20.754 python[11192:170319] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-04 11:32:20.754 python[11192:170319] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [6]:
## Ik model

# initialize model
ik = CircuitTemplate.from_yaml("/Users/utilizator/Desktop/THESIS CODE/gast_paper/config/ik_mf/ik")

# update parameters
ik.update_var(node_vars={'p/ik_op/C': C, 'p/ik_op/k': k, 'p/ik_op/v_r': v_r, 'p/ik_op/v_t': v_t, 'p/ik_op/Delta': Delta,
                         'p/ik_op/d': d, 'p/ik_op/a': a, 'p/ik_op/b': b, 'p/ik_op/tau_s': tau_s, 'p/ik_op/g': g,
                         'p/ik_op/E_r': E_r})

# run simulation
res_mf_ik = ik.run(simulation_time=T, step_size=dt, sampling_step_size=dts, solver='euler',
                outputs={'s': 'p/ik_op/s'}, inputs={'p/ik_op/I_ext': inp[:, 0]})

Compilation Progress
--------------------
	(1) Translating the circuit template into a networkx graph representation...
		...finished.
	(2) Preprocessing edge transmission operations...
		...finished.
	(3) Parsing the model equations into a compute graph...
		...finished.
	Model compilation was finished.
Simulation Progress
-------------------
	 (1) Generating the network run function...
	 (2) Processing output variables...
		...finished.
	 (3) Running the simulation...
		...finished after 19.318616430999995s.


## Plot results

In [ ]:
# Plot results
t = res_mf_ik.index
plt.figure(figsize=(10, 6))
plt.plot(t, res_mf_adex['s'], label='AdEx Model')
plt.plot(t, res_mf_ik['s'], label='Ik Model')
plt.xlabel('Time (ms)')
plt.ylabel('Synaptic Activation (s)')
plt.title('Mean-Field Model Results')
plt.legend()
plt.show()

: 

## Plot for rate output


In [ ]:
## AdEx model

# initialize model
adex = CircuitTemplate.from_yaml("/Users/utilizator/Desktop/THESIS CODE/gast_paper/config/adex_mf/adex")

# update parameters --- TODO!
adex.update_var(node_vars={'p/adex_op/C': C_adex,  'p/adex_op/v_r': v_r_adex, 'p/adex_op/v_t': v_t, 'p/adex_op/delta_T': delta_T,
                         'p/adex_op/a': a_adex, 'p/adex_op/b': b_adex, 'p/adex_op/tau_s': tau_s, 'p/adex_op/g': g_adex,
                         'p/adex_op/E_r': E_r, 'p/adex_op/tau_w': tau_w, 'p/adex_op/delta_v': Delta})

# run simulation
res_mf_adex = adex.run(simulation_time=T, step_size=dt, sampling_step_size=dts, solver='euler',
                outputs={'r': 'p/adex_op/s'}, inputs={'p/adex_op/I_ext': inp[:, 0]})



## Ik model

# initialize model
ik = CircuitTemplate.from_yaml("/Users/utilizator/Desktop/THESIS CODE/gast_paper/config/ik_mf/ik")

# update parameters
ik.update_var(node_vars={'p/ik_op/C': C, 'p/ik_op/k': k, 'p/ik_op/v_r': v_r, 'p/ik_op/v_t': v_t, 'p/ik_op/Delta': Delta,
                         'p/ik_op/d': d, 'p/ik_op/a': a, 'p/ik_op/b': b, 'p/ik_op/tau_s': tau_s, 'p/ik_op/g': g,
                         'p/ik_op/E_r': E_r})

# run simulation
res_mf_ik = ik.run(simulation_time=T, step_size=dt, sampling_step_size=dts, solver='euler',
                outputs={'r': 'p/ik_op/s'}, inputs={'p/ik_op/I_ext': inp[:, 0]})

In [ ]:
# PLOT
# Plot results
t = res_mf_ik.index
plt.figure(figsize=(10, 6))
plt.plot(t, res_mf_adex['r'], label='AdEx Model')
plt.plot(t, res_mf_ik['r'], label='Ik Model')
plt.xlabel('Time (ms)')
plt.ylabel('Rate (s)')
plt.title('Mean-Field Model Results')
plt.legend()
plt.show()